### Init ray

In [1]:
import ray
ray.init(address="auto", ignore_reinit_error=True)

2020-12-04 15:05:02,654	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 192.168.8.51:6379


{'node_ip_address': '192.168.8.51',
 'raylet_ip_address': '192.168.8.51',
 'redis_address': '192.168.8.51:6379',
 'object_store_address': '/tmp/ray/session_2020-12-04_15-00-53_639169_14044/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-12-04_15-00-53_639169_14044/sockets/raylet',
 'webui_url': '192.168.8.51:8265',
 'session_dir': '/tmp/ray/session_2020-12-04_15-00-53_639169_14044',
 'metrics_export_port': 51342}

### Load default config

In [2]:
# load defaulft config
import yaml
import os
import time
import pandas as pd
import random

config_path = './configs/Active_v0.yml'

with open(config_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    config = yaml.load(file, Loader=yaml.FullLoader)

# create base dir and gr
if os.path.exists(config["PROJECT"]["project_dir"]) is False:
    os.mkdir(config["PROJECT"]["project_dir"])

if os.path.exists(config["PROJECT"]["group_dir"]) is False:
    os.mkdir(config["PROJECT"]["group_dir"])

### Get the data to annotate

For DataNet we should pass the list of the annotated images

In [3]:
# Get the data to annotate

#############################################################################################
# LOAD DATA
#############################################################################################
from data_utils import CIFAR10Data
# Load data
cifar10_data = CIFAR10Data()
num_classes = len(cifar10_data.classes)
x_train, y_train, x_test, y_test = cifar10_data.get_data(subtract_mean=True)

/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/reminiz/ReminizML2/python_env/lib/pyt

CIFAR10 Training data shape: (50000, 32, 32, 3)
CIFAR10 Training label shape (50000, 1)
CIFAR10 Test data shape (10000, 32, 32, 3)
CIFAR10 Test label shape (10000, 1)


In [4]:
indices = list(range(len(x_train)))
random.shuffle(indices)
labeled_set = indices[:config["RUNS"]["ADDENDUM"] ]
unlabeled_set = indices[config["RUNS"]["ADDENDUM"] :]

In [5]:
# test with all the images
NUM_IMAGES_TEST = len(x_test)
# Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
test_set = list(range(NUM_IMAGES_TEST))

In [6]:
config["NETWORK"]["INPUT_SIZE"] =  x_train[0].shape[0]
config["NETWORK"]["CLASSES"] = cifar10_data.classes

In [7]:
config

{'ACTIVE_ALGO': {'LOSSLEARNING': 1.0},
 'DATASET': {'height_shift_range': 4,
  'horizontal_flip': True,
  'width_shift_range': 4},
 'NETWORK': {'CLASSES': ['plane',
   'car',
   'bird',
   'cat',
   'deer',
   'dog',
   'frog',
   'horse',
   'ship',
   'truck'],
  'INPUT_SIZE': 32,
  'MARGIN': 1.0,
  'embedding_size': 128},
 'PROJECT': {'Backbone': 'resnet18',
  'dataset_name': 'CIFAR',
  'group': 'Active_Learning_v0',
  'group_dir': '/mnt/Ressources/Andres/Temp_active/runs/Active_Learning_v0',
  'project': 'Active_Learning_CIFAR',
  'project_dir': '/mnt/Ressources/Andres/Temp_active/runs',
  'source': 'CIFAR'},
 'RUNS': {'ADDENDUM': 1000,
  'CYCLES': 10,
  'SUBSET': -1,
  'TRIALS': 1,
  'test_each': 5},
 'TEST': {'batch_size': 128},
 'TRAIN': {'Data_augementation': True,
  'EPOCH_SLIT': 80,
  'EPOCH_WARMUP': 2,
  'EPOCH_WHOLE': 120,
  'MILESTONES': [160],
  'batch_size': 128,
  'gamma': 0.1,
  'lr': 0.001,
  'start_epoch': 0,
  'transfer_weight_path': False,
  'w_c_loss': 1.0,
  'w_l

### Set Configuration from the data

In [8]:
from train_agent_cifar import Active_Learning_train
from inference_agent_cifar import Active_Learning_inference

In [9]:
import os


In [ ]:
num_run = 0
for num_run in range(6,10):
    if num_run==0:
        initial_weight_path = False
    else:
        initial_weight_path = os.path.join(config['PROJECT']['group_dir'],'Stage_'+str(num_run-1),'checkpoint','epoch200.ckpt-200')
        
        
    NetworkActor =  Active_Learning_train.remote(config, labeled_set, test_set,  num_run, initial_weight_path)
    NetworkActor.start_training.remote()
    
    # Wait util the model is training
    while True:
        time.sleep(10)
        try:
            progress_id = NetworkActor.isTraining.remote()
            response = ray.get(progress_id)
            break
        except:
            pass
        
    # wait until the model finish training
    while True:
        time.sleep(10)
        progress_id = NetworkActor.isTraining.remote()
        response = ray.get(progress_id)
        if not response:
            break
    
    NetworkActor.__ray_terminate__.remote()
    
    del NetworkActor
    
    if config["TRAIN"]["w_l_loss"] > 0:
        
        weight_file = os.path.join(config['PROJECT']['group_dir'],'Stage_'+str(num_run),'checkpoint','epoch200.ckpt-200')
        
        AL_inference = Active_Learning_inference.remote( config, unlabeled_set, num_run, weight_file)
        AL_inference.evaluate.remote()

        run_dir   = os.path.join(config["PROJECT"]["group_dir"],"Stage_"+str(num_run))
        evaluation_folder = os.path.join(run_dir, 'evaluation')
        ordered_indexes   = os.path.join(evaluation_folder, "ordered_indexes.csv")

        # wait the file qith the scores is generated
        while True:
            time.sleep(10)
            if os.path.isfile(ordered_indexes):
                break

        # read the scores file and create the new labeled set and unlabeled set to repeat the trainig
        pd_ordered_indexes = pd.read_csv(ordered_indexes)
        new_annotated_data = list(pd_ordered_indexes.iloc[:config["RUNS"]["ADDENDUM"]]['indexes'].to_numpy())
        labeled_set += new_annotated_data
        unlabeled_set =  list(pd_ordered_indexes.iloc[config["RUNS"]["ADDENDUM"]:]['indexes'].to_numpy())
    else :
        num_images = (num_run+2)*config["RUNS"]["ADDENDUM"]
        labeled_set = indices[: num_images]
        unlabeled_set = indices[num_images :]

(pid=13443, ip=192.168.8.54) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=13443, ip=192.168.8.54)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=13443, ip=192.168.8.54) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=13443, ip=192.168.8.54)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=13443, ip=192.168.8.54) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, i

(pid=13443, ip=192.168.8.54) CIFAR10 Training data shape: (50000, 32, 32, 3)
(pid=13443, ip=192.168.8.54) CIFAR10 Training label shape (50000, 1)
(pid=13443, ip=192.168.8.54) CIFAR10 Test data shape (10000, 32, 32, 3)
(pid=13443, ip=192.168.8.54) CIFAR10 Test label shape (10000, 1)


(pid=13443, ip=192.168.8.54) WARNING:tensorflow:From /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:429: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
(pid=13443, ip=192.168.8.54) Instructions for updating:
(pid=13443, ip=192.168.8.54) tf.py_func is deprecated in TF V2. Instead, use
(pid=13443, ip=192.168.8.54)     tf.py_function, which takes a python function which manipulates tf eager
(pid=13443, ip=192.168.8.54)     tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
(pid=13443, ip=192.168.8.54)     an ndarray (just call tensor.numpy()) but having access to eager tensors
(pid=13443, ip=192.168.8.54)     means `tf.py_function`s can use accelerators such as GPUs as well as
(pid=13443, ip=192.168.8.54)     being differentiable using a gradient tape.
(pid=13443, ip=192.168.8.54)     
(pid=13443, ip=192.168.8.54) WARNING:tensorflow:From /home/reminiz/

(pid=13443, ip=192.168.8.54) AL_Train_6 Path /mnt/Ressources/Andres/Temp_active/runs/Active_Learning_v0/Stage_5/checkpoint/epoch200.ckpt-200
(pid=13443, ip=192.168.8.54) AL_Train_6 => Restoring weights from: /mnt/Ressources/Andres/Temp_active/runs/Active_Learning_v0/Stage_5/checkpoint/epoch200.ckpt-200 ... 
(pid=13443, ip=192.168.8.54) AL_Train_6 self.steps_per_epoch 8


(pid=13443, ip=192.168.8.54) 2020-12-04 15:05:36.900248: I tensorflow/stream_executor/dso_loader.cc:152] successfully opened CUDA library libcublas.so.10.0 locally


...

(pid=13445, ip=192.168.8.54) WARNING:tensorflow:From /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:429: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
(pid=13445, ip=192.168.8.54) Instructions for updating:
(pid=13445, ip=192.168.8.54) tf.py_func is deprecated in TF V2. Instead, use
(pid=13445, ip=192.168.8.54)     tf.py_function, which takes a python function which manipulates tf eager
(pid=13445, ip=192.168.8.54)     tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
(pid=13445, ip=192.168.8.54)     an ndarray (just call tensor.numpy()) but having access to eager tensors
(pid=13445, ip=192.168.8.54)     means `tf.py_function`s can use accelerators such as GPUs as well as
(pid=13445, ip=192.168.8.54)     being differentiable using a gradient tape.
(pid=13445, ip=192.168.8.54)     
(pid=13445, ip=192.168.8.54) WARNING:tensorflow:From /home/reminiz/

(pid=25583, ip=192.168.8.53) AL_Train_7 Epoch:  8 / 201 || Time: 2020-12-04 15:14:58 || Accuracy: 91.12 || MAE LossNet: 0.62 || 


(pid=13445, ip=192.168.8.54) 2020-12-04 15:15:00.043385: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=13445, ip=192.168.8.54) 2020-12-04 15:15:00.078262: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3300095000 Hz
(pid=13445, ip=192.168.8.54) 2020-12-04 15:15:00.078825: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x69d0740 executing computations on platform Host. Devices:
(pid=13445, ip=192.168.8.54) 2020-12-04 15:15:00.078849: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
(pid=13445, ip=192.168.8.54) 2020-12-04 15:15:00.149895: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x69d5dc0 executing computations on platform CUDA. Devices:
(pid=13445, ip=192.168.8.54) 2020-12-04 15:15:00.149935: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0)

(pid=13445, ip=192.168.8.54) AL_Test_7 Restoring from /mnt/Ressources/Andres/Temp_active/runs/Active_Learning_v0/Stage_7/checkpoint/epoch5.ckpt-5
(pid=25583, ip=192.168.8.53) AL_Train_7 Epoch:  9 / 201 || Time: 2020-12-04 15:15:00 || Accuracy: 91.43 || MAE LossNet: 0.62 || 


(pid=13445, ip=192.168.8.54) 2020-12-04 15:15:02.403161: I tensorflow/stream_executor/dso_loader.cc:152] successfully opened CUDA library libcublas.so.10.0 locally
